# TP reconnaissance d'images

In [1]:
# img
import cv2
from skimage import io
import numpy as np
from matplotlib import pyplot as plt

In [2]:
# files
import glob
from os import mkdir

In [3]:
# miscellaneous
import math
from time import time, localtime, strftime as time_tuple_to_str, gmtime as time_float_to_tuple
from tqdm import tqdm

## Retour sur SIFT

On va commencer avec une seule image:
    

In [29]:
# Adaptez la variable 'basedir' à votre cas
basedir= r'''C:\Users\Romain\Documents\SSII\TP noté images\raw-img''' + '\\'

In [30]:
################################################################
  ######## ################ EXAMPLES ################ ########
################################################################

In [31]:
# #img = io.imread(basedir+"train/farfalla/OIP-Zq_q8o8cw50F1x6S7AmlsQHaFi.jpeg")
# #img = io.imread(basedir+"train/farfalla/eb35b30729f1073ed1584d05fb1d4e9fe777ead218ac104497f5c97faeebb5bb_640.png")
# img = io.imread(basedir+"train/ragno/eb3cb3082af0073ed1584d05fb1d4e9fe777ead218ac104497f5c97ca5edb3bd_640.png")
# plt.imshow(img)
# print(img.shape)

In [32]:
# gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
# sift = cv2.SIFT_create()
# now = time()
# kp,des = sift.detectAndCompute(gray,None)
# duration = time() - now
# print("durée = %.2f s" %duration)
# cv2.drawKeypoints(gray,kp,img,flags=cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS)
# plt.imshow(gray, cmap='gray')

In [33]:
################################################################
######## ################ EXAMPLES END ################ ########
################################################################

In [116]:
def build_SIFTs(listImages = [], n = 20, debug = True, SIFT_nbr_per_img = [], reverse = False):
    sifts = np.empty(shape=(0, 128), dtype=int)
    if not reverse:
        if n > 0:
            img_list = listImages[:int(n)]
        elif n == -1:
            img_list = listImages[:]
        else:
            img_list = listImages[:20]
    else:
        if n > 0:
            img_list = listImages[-int(n):]
        elif n == -1:
            img_list = listImages[:]
        else:
            img_list = listImages[-20:]


    for pathImg in (tqdm(img_list, position=0, leave=True) if debug else img_list):
        img = io.imread(pathImg)
        gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
        sift = cv2.SIFT_create()
        # kp, des = sift.detectAndCompute(gray,None)
        des = sift.detectAndCompute(gray,None)[1]

        sifts = np.append(sifts, des, axis=0)
        
        SIFT_nbr_per_img.append(len(des))

    return sifts


def build_BoF(img_SIFTs, km):
    clusters_found = list(km.predict(img_SIFTs))
    return (1/len(img_SIFTs))*np.array([clusters_found.count(i) for i in range(len(km.cluster_centers_))])


def build_all_BoF(sifts, SIFT_nbr_per_img, km):
    
    result = []
    
    previous_sift_nbr = 0
    for sift_nbr_for_img in SIFT_nbr_per_img:
        result.append(build_BoF(sifts[previous_sift_nbr: previous_sift_nbr + sift_nbr_for_img], km))
        
        previous_sift_nbr += sift_nbr_for_img
    
    return result


def less_quick_build_all_BoF(classes, basedir, km, n):
    ''' Equivalent, but much slower'''

    images = [img for cl in classes for img in glob.glob(basedir + cl + '/*')[:n]]
    
    k = len(km.cluster_centers_)

    SIFT_nbr_per_img = [] # To transform the BoF of each image from int value (count) to percentage

    all_SIFTs = build_SIFTs(images, n = -1, debug = False, SIFT_nbr_per_img = SIFT_nbr_per_img)
    
    clusters_found = list(km.predict(all_SIFTs))
    
    
    result = np.empty(shape=(0, k), dtype=float)
    
    previous_SIFTs_nbr = 0
    for img_i in range(len(images)):
        actual_SIFTs_nbr = SIFT_nbr_per_img[img_i]

        this_img_BoF = np.array([clusters_found[previous_SIFTs_nbr : previous_SIFTs_nbr + actual_SIFTs_nbr].count(i)\
                                         for i in range(k)], dtype = float) # BoF
        this_img_BoF /= actual_SIFTs_nbr # fraction

        result = np.append(result, [this_img_BoF], axis = 0)
        previous_SIFTs_nbr += actual_SIFTs_nbr
            
    return result

In [35]:
def time_str():
    return time_tuple_to_str('''%Y.%m.%d %Hh%Mm%Ss''', localtime())

In [36]:
# Methods to save data

pickle_folder = 'pickle_saves\\'
pickle_extension = '.pkl'

def make_pkl_dir():
    global pickle_folder
    try:
        mkdir(basedir + pickle_folder)
    except FileExistsError:
        pass

def save(array, name = None):
    global basedir, pickle_folder, pickle_extension

    if name == None:
        name = time_str() + ' - array_saved' + pickle_extension
    elif not name.endswith(pickle_extension):
        name += pickle_extension
    
    make_pkl_dir()
    
    array.dump(basedir + pickle_folder + name)
    return name

def load(name):
    global basedir, pickle_folder

    file_name = basedir + pickle_folder + name
    
    make_pkl_dir()
    try:
        array = np.load(file_name, allow_pickle = True)
    except:
        print(f'Somthing went wrong while trying to retrieve data from "{file_name}"')
        return np.array([])
    else:
        return array

Que contiennent les variables kp et des ? Quelles sont les dimensions ?

In [37]:
#print('len(kp):', len(kp))
#print('type(kp[0]):', type(kp[0]))
#print()
#print('des[0]:', des[0])
#print('type(des[0]):', type(des[0]))
#print('len(des[0]):', len(des[0]))
#print()
#print('des:', des)
#print('type(des):', type(des))
#print('len(des):', len(des))
#print('des.shape:', des.shape)

## Chargement de vos classes d'images

In [38]:
baseTrain = basedir + 'train/'
baseTest = basedir + 'test/'
classes = ['pecora','ragno']

In [111]:
nb = {} # nombre d'images par classe
SIFTs = {} # SIFTs par classe
test_SIFTs = {}

SIFT_nbr_per_img_per_cl = {}
test_SIFT_nbr_per_img_per_cl = {}

n = 20 # échantillon d'images

for cl in classes:
    SIFT_nbr_per_img_per_cl[cl] = []
    test_SIFT_nbr_per_img_per_cl[cl] = []
    
    prev = time()
    listImages = glob.glob(baseTrain + cl + '/*')
    print('class:', cl, ':', len(listImages), 'images.') 
    nb[cl] = len(listImages)
    SIFTs[cl] = build_SIFTs(listImages, n = n, SIFT_nbr_per_img = SIFT_nbr_per_img_per_cl[cl])
    test_SIFTs[cl] = build_SIFTs(listImages, n = n_test, SIFT_nbr_per_img = test_SIFT_nbr_per_img_per_cl[cl], reverse = True)

    print('classe {}: {:.2f} s\n'.format(cl, time() - prev))

print('total number of images:', sum(nb.values()))
print('longueur des SIFTs')
print(*(f'{cl}: ' + str(len(SIFTs[cl])) for cl in classes), sep = ', ')
print(SIFT_nbr_per_img_per_cl)

  0%|          | 0/20 [00:00<?, ?it/s]

class: pecora : 1456 images.


  0%|          | 0/20 [00:00<?, ?it/s]

classe pecora: 6.62 s

class: ragno : 3856 images.


100%|██████████| 20/20 [00:02<00:00,  8.19it/s]

classe ragno: 8.00 s

total number of images: 5312
longueur des SIFTs
pecora: 36143, ragno: 31690
{'pecora': [3409, 1212, 1940, 717, 400, 1458, 2337, 1936, 1347, 1844, 2155, 2154, 1895, 1396, 2304, 545, 1676, 1592, 2185, 3641], 'ragno': [346, 2633, 796, 1172, 172, 1032, 1932, 2583, 3405, 3147, 2200, 1010, 853, 4022, 1837, 836, 1140, 1263, 410, 901]}


In [110]:
print([sum(i) for i in SIFT_nbr_per_img_per_cl.values()])

[36143, 31690]


In [ ]:
for cl in classes:
    print(cl + ':', SIFTs[cl].shape)

## Sac de descripteurs (BoF)

In [57]:
k_min = 20
k_max = 25

In [58]:
from sklearn.cluster import MiniBatchKMeans#, KMeans

In [89]:
# merge sifts from each class in one list
all_SIFTs = np.empty(shape=(0, 128), dtype=int)
all_test_SIFTs = np.empty(shape=(0, 128), dtype=int)

for cl in classes:
    all_SIFTs = np.append(all_SIFTs, SIFTs[cl], axis = 0)
    all_test_SIFTs = np.append(all_test_SIFTs, test_SIFTs[cl], axis = 0)

In [74]:
################
### Single k ###
################

In [125]:
k = 50

K_Means = MiniBatchKMeans(n_clusters = k).fit(all_SIFTs)
test_K_means = MiniBatchKMeans(n_clusters = k).fit(all_test_SIFTs)

labels = K_Means.labels_ # for each SIFT, a cluster label

# print({i: list(labels).count(i) for i in range(k)})


#KMeans_dic = {} # KMeans par classe
#labels = {} # labels par classe
#for cl in classes:
#    KMeans_dic[cl] = MiniBatchKMeans(n_clusters = k).fit(SIFTs[cl])
#    labels[cl] = KMeans_dic[cl].labels_ # for each SIFT, a cluster label
#    print(labels[cl])
#    print({i: list(labels[cl]).count(i) for i in range(k)})

In [78]:
print(labels.shape)
print(all_SIFTs.shape)

(67833,)
(67833, 128)


In [122]:
SIFTs_concat = []
SIFTs_nbr_per_image = []
test_SIFTs_concat = []
test_SIFTs_nbr_per_image = []

for cl in SIFTs:
    SIFTs_concat.extend(SIFTs[cl])
    SIFTs_nbr_per_image.extend(SIFT_nbr_per_img_per_cl[cl])
    test_SIFTs_concat.extend(test_SIFTs[cl])
    test_SIFTs_nbr_per_image.extend(test_SIFT_nbr_per_img_per_cl[cl])

In [123]:
prev = time()

BoFs_both_classes = build_all_BoF(SIFTs_concat, SIFTs_nbr_per_image, K_Means)
test_BoFs_both_classes = build_all_BoF(test_SIFTs_concat, test_SIFTs_nbr_per_image, K_Means)

print('known lenght:', len(BoFs_both_classes[0]))
print(BoFs_both_classes[0])

print(f'{time() - prev}s elapsed')

known lenght: 50
[0.02112056 0.00733353 0.0164271  0.0199472  0.02757407 0.02288061
 0.01789381 0.02053388 0.02229393 0.00792021 0.0149604  0.01408038
 0.01056028 0.03344089 0.02229393 0.01760047 0.01056028 0.02669405
 0.02112056 0.02552068 0.0313875  0.02112056 0.01349369 0.0214139
 0.01818715 0.02288061 0.02258727 0.02024054 0.01701379 0.01232033
 0.01114696 0.01906718 0.03842769 0.02552068 0.024934   0.02170725
 0.01232033 0.01760047 0.01848049 0.01701379 0.02229393 0.02317395
 0.02757407 0.01525374 0.0214139  0.01290701 0.01672045 0.02024054
 0.02229393 0.03050748]
2.483920097351074s elapsed


In [80]:
prev = time()

BoFs_both_classes = less_quick_build_all_BoF(classes, baseTrain, K_Means, n)

print('known lenght:', len(BoFs_both_classes[0]))
print(BoFs_both_classes[0])

print(f'{time() - prev}s elapsed')

known lenght: 50
[0.02170725 0.00880023 0.03461426 0.0214139  0.01114696 0.02610736
 0.02581402 0.02669405 0.01965386 0.0313875  0.02053388 0.01730713
 0.01877383 0.02522734 0.00968026 0.00674685 0.01936052 0.02698739
 0.02786741 0.01818715 0.01232033 0.         0.00909358 0.02288061
 0.02112056 0.00909358 0.01144031 0.01349369 0.02728073 0.02288061
 0.01672045 0.02376063 0.01848049 0.02874743 0.01613376 0.02258727
 0.02258727 0.02288061 0.02112056 0.03021414 0.0199472  0.01056028
 0.02200059 0.02962746 0.02581402 0.03050748 0.01965386 0.01554708
 0.01789381 0.01760047]
10.566467046737671s elapsed


In [ ]:
################
# Single k end #
################

In [ ]:
## All k

In [118]:
print('K_Means')
K_means_dico = {}
test_K_means_dico = {}

prev = time()

for k in range(k_min, k_max):
    K_means_dico[k] = MiniBatchKMeans(n_clusters = k).fit(all_SIFTs)
    test_K_means_dico[k] = MiniBatchKMeans(n_clusters = k).fit(all_test_SIFTs)
    
    print(f'{k+1}/100 in {int(time() - prev)}s', end = '\r')

K_Means


In [119]:
print('BoFs')
BoFs_both_classes_dico = {}
test_BoFs_both_classes_dico = {}

prev = time()

for k in range(k_min, k_max):
    BoFs_both_classes_dico[k] = build_all_BoF(classes, baseTrain, K_means_dico[k], n)
    test_BoFs_both_classes_dico[k] = build_all_BoF(classes, baseTest, test_K_means_dico[k], n)

    print(f'{k+1}/100 in {int(time() - prev)}s', end = '\r')

BoFs


TypeError: build_all_BoF() takes 3 positional arguments but 4 were given

## Classification par regression logistique

In [50]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score

In [126]:
log_reg = LogisticRegression()

groundTruth = [0]*n + [1]*n
log_reg.fit(BoFs_both_classes, (groundTruth))

LogisticRegression()

In [127]:
resTrain = log_reg.predict(test_BoFs_both_classes)
scoreTrain = f1_score((groundTruth), resTrain)
print(f'train score pour k = {k}:', scoreTrain)

train score pour k = 50: 0.41379310344827586


In [92]:
print('scores train')
scoresTrain_dico = {}
log_reg_dico = {}

groundTruth = [0]*n + [1]*n

for k in range(k_min, k_max):
    log_reg = LogisticRegression()
    log_reg.fit(BoFs_both_classes_dico[k], groundTruth)
    print(len(BoFs_both_classes_dico[k][0]))

    scoresTrain_dico[k] = f1_score(groundTruth, log_reg.predict(test_BoFs_both_classes_dico[k]))

    log_reg_dico[k] = log_reg
    
print(scoresTrain_dico)

scores train
20
21
22
23
24
{20: 0.45161290322580644, 21: 0.46808510638297873, 22: 0.5555555555555556, 23: 0.6071428571428571, 24: 0.5454545454545455}


In [123]:
best_k = max(scoresTrain_dico.values())
for k in scoresTrain_dico:
    if scoresTrain_dico[k] == best_k:
        best_k = k
        break
else:
    raise ValueError('k were not found, it is a problem')

print(best_k)

2


## test dataset

In [125]:
for cl in classes:
    print(f'classe "{cl}"')
    for img in glob.glob(baseTest + cl + '/*')[:n]:
        
        bof_test = build_BoF(img, K_means_dico[best_k])
        
        if f1_score(groundTruth, log_reg_dico[k].predict([bof_test])) >= 0.5:
            print('class found:', cl)
        else:
            print('class not found (not ' + cl + ')')

classe "pecora"
[array([0.38040557, 0.61959443])]


ValueError: X has 2 features per sample; expecting 50